In [1]:
import os
%pwd
os.chdir('../')
%pwd

'd:\\Projects\\ML\\End-To-End ML Projects\\TextSummarizer'

In [10]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
from src.textSummarizer.entity import (DataIngestionConfig,
                                      DataTransformationConfig,
                                      ModelTrainerConfig,
                                      ModelEvalConfig)

class ConfigurationManager:
    def __init__(self,config_file__path=CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file__path)
        self.params = read_yaml(params_file_path)
    
    def get_model_eval_config(self) -> ModelEvalConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])
        
        model_eval_config = ModelEvalConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )
        
        return model_eval_config

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
from src.textSummarizer.entity import ModelEvalConfig
import torch
import evaluate
import pandas as pd
from tqdm import tqdm

class ModelEvaluation:
    def __init__(self,config:ModelEvalConfig):
        self.config = config
    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]
    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer,
                               batch_size=16, device="cuda",
                               column_text="article",
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

            # Finally, we decode the generated texts,
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        
        rouge_metric = evaluate.load('rouge')
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        
        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, device=device ,column_text = 'dialogue', column_summary= 'summary'
        )

        # Directly use the scores without accessing fmeasure or mid
        rouge_dict = {rn: score[rn] for rn in rouge_names}

        # Convert the dictionary to a DataFrame for easy visualization
        df = pd.DataFrame(rouge_dict, index=[f'pegasus'])
        df.to_csv(self.config.metric_file_name,index=False)
           
         

In [12]:
config = ConfigurationManager()
model_eval_config = config.get_model_eval_config()
model_eval = ModelEvaluation(config=model_eval_config)
model_eval.evaluate()

[2025-08-31 22:11:42,787 : INFO : common : YAML file config\config.yaml loaded successfully]
[2025-08-31 22:11:42,790 : INFO : common : YAML file params.yaml loaded successfully]
[2025-08-31 22:11:42,792 : INFO : common : Creater directory at artifacts/model_evaluation]


100%|██████████| 5/5 [16:43<00:00, 200.64s/it]


[2025-08-31 22:28:44,650 : INFO : rouge_scorer : Using default tokenizer.]
